# Лабораторная работа №4: Проведение исследований со случайным лесом

## 1. Выбор начальных условий

Был проведен в ЛР №1.

In [ ]:
!pip install kagglehub scikit-learn numpy pandas matplotlib seaborn

In [29]:
import kagglehub
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [30]:
# Скачивание Date Fruit Dataset
date_fruit_path = kagglehub.dataset_download("muratkokludataset/date-fruit-datasets")

# Чтение Excel-файла
date_fruit_data = pd.read_excel(f"{date_fruit_path}/Date_Fruit_Datasets/Date_Fruit_Datasets.xlsx")

In [31]:
# Скачивание Concrete Compressive Strength
concrete_strength_path = kagglehub.dataset_download("niteshyadav3103/concrete-compressive-strength")

# Чтение CSV-файла
concrete_data = pd.read_csv(f"{concrete_strength_path}/Concrete Compressive Strength.csv")

## 2. Создание бейзлайна и оценка качества

In [32]:
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, f1_score, root_mean_squared_error, r2_score, make_scorer
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.pipeline import Pipeline

Разделим датасет для классификации на обучающую и тестовую выборки

In [33]:
# Разделение на признаки и целевую переменную
X_class = date_fruit_data.drop(columns=['Class'])
y_class = date_fruit_data['Class']

# Разделение на обучающую и тестовую выборки
X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(
    X_class, y_class, test_size=0.2, random_state=42, stratify=y_class
)

# Преобразование целевой переменной
label_encoder = LabelEncoder()
y_train_class = label_encoder.fit_transform(y_train_class)
y_test_class = label_encoder.transform(y_test_class)

Аналогично разделим датасет для регрессии

In [34]:
# Разделение на признаки и целевую переменную
X_reg = concrete_data.drop(columns=['Concrete compressive strength '])
y_reg = concrete_data['Concrete compressive strength ']

# Разделение на обучающую и тестовую выборки
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42
)

Обучим модели для классификации и регрессии из Sklearn и оценим их

In [35]:
rf_classifier = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
rf_classifier.fit(X_train_class, y_train_class)

y_pred_class = rf_classifier.predict(X_test_class)

accuracy_rf = accuracy_score(y_test_class, y_pred_class)
f1_rf = f1_score(y_test_class, y_pred_class, average='weighted')

print(f"Random Forest Classifier - Accuracy: {accuracy_rf:.4f}, F1-Score: {f1_rf:.4f}")

Random Forest Classifier - Accuracy: 0.9222, F1-Score: 0.9214


In [36]:
rf_regressor = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
rf_regressor.fit(X_train_reg, y_train_reg)

y_pred_reg = rf_regressor.predict(X_test_reg)

rmse_rf = root_mean_squared_error(y_test_reg, y_pred_reg)
r2_rf = r2_score(y_test_reg, y_pred_reg)

print(f"Random Forest Regressor - RMSE: {rmse_rf:.4f}, R²: {r2_rf:.4f}")

Random Forest Regressor - RMSE: 5.5397, R²: 0.8809


Итак, точность для встроенной в Sklearn модели случайного леса получилась отличной, в частности для регрессора, просто невероятной по сравнению с предыдущими ЛР (92.2% точности для классификатора, 5.53 RMSE для регрессора). Попробуем улучшить бейзлайн.

## 3. Улучшение бейзлайна

Будем перебирать гиперпараметры случайного леса, такие, как n_estimators, max_depth, min_samples_split, min_samples_leaf

In [37]:
from sklearn.model_selection import GridSearchCV

# Сокращенный диапазон гиперпараметров
param_grid_classifier = {
    'n_estimators': [50, 100],
    'max_depth': [5, 10, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
}

param_grid_regressor = {
    'n_estimators': [50, 100],
    'max_depth': [5, 10, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
}

# === Подбор гиперпараметров для RandomForestClassifier ===
print("=== Random Forest Classifier - Hyperparameter Tuning ===")
grid_search_rf_classifier = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_grid=param_grid_classifier,
    scoring='accuracy',  # Используем Accuracy вместо F1-Score
    cv=3,                # Уменьшаем количество фолдов
    verbose=1
)

grid_search_rf_classifier.fit(X_train_class, y_train_class)

# Лучшие параметры и результат
best_params_rf_classifier = grid_search_rf_classifier.best_params_
best_score_rf_classifier = grid_search_rf_classifier.best_score_

print(f"Лучшие параметры для Random Forest Classifier: {best_params_rf_classifier}")
print(f"Лучший Accuracy на кросс-валидации: {best_score_rf_classifier:.4f}")

# === Подбор гиперпараметров для RandomForestRegressor ===
print("\n=== Random Forest Regressor - Hyperparameter Tuning ===")
grid_search_rf_regressor = GridSearchCV(
    estimator=RandomForestRegressor(random_state=42),
    param_grid=param_grid_regressor,
    scoring='neg_root_mean_squared_error',
    cv=3,                # Уменьшаем количество фолдов
    verbose=1
)

grid_search_rf_regressor.fit(X_train_reg, y_train_reg)

# Лучшие параметры и результат
best_params_rf_regressor = grid_search_rf_regressor.best_params_
best_score_rf_regressor = -grid_search_rf_regressor.best_score_

print(f"Лучшие параметры для Random Forest Regressor: {best_params_rf_regressor}")
print(f"Лучший RMSE на кросс-валидации: {best_score_rf_regressor:.4f}")


=== Random Forest Classifier - Hyperparameter Tuning ===
Fitting 3 folds for each of 24 candidates, totalling 72 fits
Лучшие параметры для Random Forest Classifier: {'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 100}
Лучший Accuracy на кросс-валидации: 0.8719

=== Random Forest Regressor - Hyperparameter Tuning ===
Fitting 3 folds for each of 24 candidates, totalling 72 fits
Лучшие параметры для Random Forest Regressor: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Лучший RMSE на кросс-валидации: 5.2606


Обучим модели с лучшими гиперпараметрами

In [38]:
# RandomForestClassifier с подобранными параметрами
best_rf_classifier = RandomForestClassifier(
    n_estimators=best_params_rf_classifier['n_estimators'],
    max_depth=best_params_rf_classifier['max_depth'],
    min_samples_split=best_params_rf_classifier['min_samples_split'],
    min_samples_leaf=best_params_rf_classifier['min_samples_leaf'],
    random_state=42
)
best_rf_classifier.fit(X_train_class, y_train_class)
y_pred_class = best_rf_classifier.predict(X_test_class)

accuracy = accuracy_score(y_test_class, y_pred_class)
f1 = f1_score(y_test_class, y_pred_class, average='weighted')

print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test F1-Score: {f1:.4f}")

# RandomForestRegressor с подобранными параметрами
best_rf_regressor = RandomForestRegressor(
    n_estimators=best_params_rf_regressor['n_estimators'],
    max_depth=best_params_rf_regressor['max_depth'],
    min_samples_split=best_params_rf_regressor['min_samples_split'],
    min_samples_leaf=best_params_rf_regressor['min_samples_leaf'],
    random_state=42
)
best_rf_regressor.fit(X_train_reg, y_train_reg)
y_pred_reg = best_rf_regressor.predict(X_test_reg)

rmse = root_mean_squared_error(y_test_reg, y_pred_reg)
r2 = r2_score(y_test_reg, y_pred_reg)

print(f"Test RMSE: {rmse:.4f}")
print(f"Test R²: {r2:.4f}")


Test Accuracy: 0.9278
Test F1-Score: 0.9281
Test RMSE: 5.4444
Test R²: 0.8850


Как видно, точность немного выросла, как для классификации, так и для регрессии.

## 4. Имплементация алгоритма машинного обучения

Напишем собственную реализацию случайного леса для классификации и регрессии, затем обучим модели на тестовых данных и сравним по качеству с реализациями из Sklearn. Будем использовать реализацию решающего дерева из ЛР №3.

In [49]:
class DecisionTreeNode:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature          # Признак для разбиения
        self.threshold = threshold      # Пороговое значение
        self.left = left                # Левое поддерево
        self.right = right              # Правое поддерево
        self.value = value              # Значение в листе (для терминальных узлов)

class DecisionTreeClassifierCustom:
    def __init__(self, max_depth=None, min_samples_split=2, min_samples_leaf=1):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.root = None

    def _gini(self, y):
        proportions = np.bincount(y) / len(y)
        return 1 - np.sum(proportions ** 2)

    def _split(self, X, y, feature, threshold):
        left_indices = np.where(X[:, feature] < threshold)[0]
        right_indices = np.where(X[:, feature] >= threshold)[0]
        return X[left_indices], y[left_indices], X[right_indices], y[right_indices]

    def _best_split(self, X, y):
        best_score = float('inf')  # Лучший критерий разбиения (минимальный)
        best_split = None

        for feature in range(X.shape[1]):  # Перебор всех признаков
            thresholds = np.unique(X[:, feature])  # Уникальные значения признака
            for threshold in thresholds:
                # Разбиение данных
                X_left, y_left, X_right, y_right = self._split(X, y, feature, threshold)

                # Проверка, что оба подмножества достаточно велики
                if len(y_left) < self.min_samples_leaf or len(y_right) < self.min_samples_leaf:
                    continue

                # Вычисление критерия Gini
                score = (len(y_left) * self._gini(y_left) + len(y_right) * self._gini(y_right)) / len(y)
                if score < best_score:
                    best_score = score
                    best_split = (feature, threshold)

        return best_split

    def _build_tree(self, X, y, depth):
        # Условие остановки
        if (len(np.unique(y)) == 1 or depth == self.max_depth or
            len(y) < self.min_samples_split):
            return DecisionTreeNode(value=np.bincount(y).argmax())

        # Найти лучшее разбиение
        best_split = self._best_split(X, y)
        if best_split is None:
            return DecisionTreeNode(value=np.bincount(y).argmax())

        feature, threshold = best_split

        # Разделение данных
        X_left, y_left, X_right, y_right = self._split(X, y, feature, threshold)
        if len(y_left) < self.min_samples_leaf or len(y_right) < self.min_samples_leaf:
            return DecisionTreeNode(value=np.bincount(y).argmax())

        left_child = self._build_tree(X_left, y_left, depth + 1)
        right_child = self._build_tree(X_right, y_right, depth + 1)

        return DecisionTreeNode(feature, threshold, left_child, right_child)

    def fit(self, X, y):
        self.root = self._build_tree(X, y, 0)

    def _predict(self, x, node):
        if node.value is not None:
            return node.value
        if x[node.feature] < node.threshold:
            return self._predict(x, node.left)
        else:
            return self._predict(x, node.right)

    def predict(self, X):
        return np.array([self._predict(x, self.root) for x in X])

class DecisionTreeRegressorCustom:
    def __init__(self, max_depth=None, min_samples_split=2, min_samples_leaf=1):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.root = None

    def _mse(self, y):
        return np.mean((y - np.mean(y)) ** 2)

    def _split(self, X, y, feature, threshold):
        left_indices = np.where(X[:, feature] < threshold)[0]
        right_indices = np.where(X[:, feature] >= threshold)[0]
        return X[left_indices], y[left_indices], X[right_indices], y[right_indices]

    def _best_split(self, X, y):
        best_score = float('inf')  # Лучший критерий разбиения (минимальный MSE)
        best_split = None

        for feature in range(X.shape[1]):  # Перебор всех признаков
            thresholds = np.unique(X[:, feature])  # Уникальные значения признака
            for threshold in thresholds:
                # Разбиение данных
                X_left, y_left, X_right, y_right = self._split(X, y, feature, threshold)

                # Проверка, что оба подмножества достаточно велики
                if len(y_left) < self.min_samples_leaf or len(y_right) < self.min_samples_leaf:
                    continue

                # Вычисление MSE
                score = (len(y_left) * self._mse(y_left) + len(y_right) * self._mse(y_right)) / len(y)
                if score < best_score:
                    best_score = score
                    best_split = (feature, threshold)

        return best_split

    def _build_tree(self, X, y, depth):
        # Условие остановки
        if (len(np.unique(y)) == 1 or depth == self.max_depth or
            len(y) < self.min_samples_split):
            return DecisionTreeNode(value=np.mean(y))

        # Найти лучшее разбиение
        best_split = self._best_split(X, y)
        if best_split is None:
            return DecisionTreeNode(value=np.mean(y))

        feature, threshold = best_split

        # Разделение данных
        X_left, y_left, X_right, y_right = self._split(X, y, feature, threshold)
        if len(y_left) < self.min_samples_leaf or len(y_right) < self.min_samples_leaf:
            return DecisionTreeNode(value=np.mean(y))

        left_child = self._build_tree(X_left, y_left, depth + 1)
        right_child = self._build_tree(X_right, y_right, depth + 1)

        return DecisionTreeNode(feature, threshold, left_child, right_child)

    def fit(self, X, y):
        self.root = self._build_tree(X, y, 0)

    def _predict(self, x, node):
        if node.value is not None:
            return node.value
        if x[node.feature] < node.threshold:
            return self._predict(x, node.left)
        else:
            return self._predict(x, node.right)

    def predict(self, X):
        return np.array([self._predict(x, self.root) for x in X])

class CustomRandomForestClassifier:
    def __init__(self, n_estimators=10, max_depth=None, min_samples_split=2, min_samples_leaf=1, max_features=None, random_state=None):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.random_state = random_state
        self.trees = []

    def _bootstrap_sample(self, X, y):
        n_samples = X.shape[0]
        indices = np.random.choice(n_samples, n_samples, replace=True)
        return X[indices], y[indices]

    def fit(self, X, y):
        np.random.seed(self.random_state)
        self.trees = []
        for _ in range(self.n_estimators):
            X_sample, y_sample = self._bootstrap_sample(X, y)
            tree = DecisionTreeClassifierCustom(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split,
                min_samples_leaf=self.min_samples_leaf
            )
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)

    def predict(self, X):
        tree_predictions = np.array([tree.predict(X) for tree in self.trees])
        return np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=tree_predictions)


class CustomRandomForestRegressor:
    def __init__(self, n_estimators=10, max_depth=None, min_samples_split=2, min_samples_leaf=1, max_features=None, random_state=None):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.random_state = random_state
        self.trees = []

    def _bootstrap_sample(self, X, y):
        n_samples = X.shape[0]
        indices = np.random.choice(n_samples, n_samples, replace=True)
        return X[indices], y[indices]

    def fit(self, X, y):
        np.random.seed(self.random_state)
        self.trees = []
        for _ in range(self.n_estimators):
            X_sample, y_sample = self._bootstrap_sample(X, y)
            tree = DecisionTreeRegressorCustom(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split,
                min_samples_leaf=self.min_samples_leaf
            )
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)

    def predict(self, X):
        tree_predictions = np.array([tree.predict(X) for tree in self.trees])
        return np.mean(tree_predictions, axis=0)

Обучим модели и оценим их качество

In [50]:
scaler = StandardScaler()
X_train_class = scaler.fit_transform(X_train_class)
X_test_class = scaler.transform(X_test_class)

custom_rf_classifier = CustomRandomForestClassifier(n_estimators=10, max_depth=5, random_state=42)
custom_rf_classifier.fit(X_train_class, y_train_class)

y_pred_custom_class = custom_rf_classifier.predict(X_test_class)

accuracy_custom = accuracy_score(y_test_class, y_pred_custom_class)
f1_custom = f1_score(y_test_class, y_pred_custom_class, average='weighted')

print(f"Custom Random Forest Classifier - Accuracy: {accuracy_custom:.4f}, F1-Score: {f1_custom:.4f}")

Custom Random Forest Classifier - Accuracy: 0.8667, F1-Score: 0.8678


In [53]:
scaler = StandardScaler()
X_train_reg = scaler.fit_transform(X_train_reg)
X_test_reg = scaler.transform(X_test_reg)

custom_rf_regressor = CustomRandomForestRegressor(n_estimators=10, max_depth=5, random_state=42)
custom_rf_regressor.fit(X_train_reg, np.array(y_train_reg))

y_pred_custom_reg = custom_rf_regressor.predict(X_test_reg)

rmse_custom = root_mean_squared_error(y_test_reg, y_pred_custom_reg)
r2_custom = r2_score(y_test_reg, y_pred_custom_reg)

print(f"Custom Random Forest Regressor - RMSE: {rmse_custom:.4f}, R²: {r2_custom:.4f}")

Custom Random Forest Regressor - RMSE: 7.4933, R²: 0.7821


Точность вручную реализованных моделей оказалась похуже, чем у библиотечных, но все равно на приемлемом уровне.